In [1]:
import os
import sys
import cv2 
import time
import json
import math
import copy
import torch
import numpy as np
import kornia as K
import kornia.feature as KF
from vidstab import VidStab

from lightglue import LightGlue, SuperPoint
from lightglue.utils import load_image, rbd
import CSRansac

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
#matcher = LightGlue(features='superpoint', depth_confidence=0.9, width_confidence=0.95).eval().to(device)
matcher = LightGlue(features='superpoint', depth_confidence=-1, width_confidence=-1).eval().to(device)
#matcher.compile(mode='reduce-overhead')

In [4]:
print(torch.__version__)
print(device)

2.1.2
cuda


In [5]:
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (640, 480))  # 필요한 경우 이미지 크기 조정
    image = K.image_to_tensor(image, False).float() / 255.0
    image = image.to(device)
    return image

In [6]:
def match_lightglue(img0, img1):
    img0 = load_image(img0)
    img1 = load_image(img1)

    # extract local features
    feats0 = extractor.extract(img0.to(device))  # auto-resize the image, disable with resize=None
    feats1 = extractor.extract(img1.to(device))
    
    # match the features
    matches01 = matcher({'image0': feats0, 'image1': feats1})
    feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]  # remove batch dimension
    
    # get results
    kpts0 = feats0["keypoints"]
    kpts1 = feats1["keypoints"]
    matches = matches01['matches']  # indices with shape (K,2)
    points0 = kpts0[matches[..., 0]]  # coordinates in img0, shape (K,2)
    points1 = kpts1[matches[..., 1]]  # coordinates in img1, shape (K,2)
        
    return {
        "points0": points0,
        "points1": points1,
    }

In [7]:
stabilizer = VidStab()

def matching_keypoints(target_img, video_img, stabilizing=False):
    # 이미지를 불러옴
    img0 = load_image(target_img, grayscale=True)
    if stabilizing == True:
        img1 = cv2.imread(video_img)
        img1 = stabilizer.stabilize_frame(img1)
        img1 = load_image(img1, grayscale=True)
    else:
        img1 = load_image(video_img , grayscale=True)

    # extract local features
    feats0 = extractor.extract(img0.to(device))  # auto-resize the image, disable with resize=None
    feats1 = extractor.extract(img1.to(device))

    # match the features
    matches01 = matcher({'image0': feats0, 'image1': feats1})
    feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]  # remove batch dimension

    # get results
    kpts0 = feats0["keypoints"]
    kpts1 = feats1["keypoints"]
    matches = matches01['matches']  # indices with shape (K,2)
    points0 = kpts0[matches[..., 0]]  # coordinates in img0, shape (K,2)
    points1 = kpts1[matches[..., 1]]  # coordinates in img1, shape (K,2)

    return {
        "points0": points0,
        "points1": points1,
    }

In [8]:
cracked_image = "CrackedImage.PNG"
video = "Roboinspect.mp4"

In [11]:
# 크랙 이미지를 통해 크랙을 탐지함
cap = cv2.VideoCapture(video)

x, y = 1408 / 2, 792 / 2

# 동영상 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 설정
out = cv2.VideoWriter('output_video.avi', fourcc, 20.0, (640, 360))  # 파일명, 프레임 속도, 해상도

img0 = cracked_image

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.resize(frame, (640, 360))
    img1 = frame
    
    # LightGlue
    results_lightglue = matching_keypoints(img0, img1, stabilizing=False)
    target_keypoint = results_lightglue["points0"].cpu().numpy()
    frame_keypoint = results_lightglue["points1"].cpu().numpy()

    # CSRansac으로 호모그래피 계산 및 mask 확인
    homography, mask = CSRansac.csransac(target_keypoint, frame_keypoint)
    
    # mask가 0.7 이상일 때 화면 중앙에 빨간 점 그리기
    if np.mean(mask) >= 0.8:
        pts = CSRansac.perspective_transform(np.array([x, y]), homography)
        #cv2.circle(frame, (int(pts[0]), int(pts[1])), 10, (0, 0, 255), -1)  # 빨간 점 그리기
        cv2.rectangle(frame, (int(pts[0])-100, int(pts[1])-100), (int(pts[0])+100, int(pts[1])+100), (0, 0, 255), 2)
        
        
    # 프레임을 동영상 파일로 저장
    out.write(frame)
    
    # 프레임 출력
    cv2.imshow("frame", frame)

    key = cv2.waitKey(5)
    if key == 27:  # ESC 키를 누르면 종료
        break

# 리소스 해제
cap.release()
out.release()  # 저장 파일 닫기
cv2.destroyAllWindows()

KeyboardInterrupt: 